### Curation Layer

#### Importing libraries

In [1]:
import import_ipynb
from pyspark.sql.functions import col,sum,max,min,when,dense_rank
from pyspark.sql.window import Window

#### Setting configs and path

In [2]:
from Projects.SHIMANO_CASE_STUDY.raw_layer.file_to_raw import read_input_datasets

importing Jupyter notebook from /Users/sahilnagpal/Desktop/coding/competitive-programming/Projects/SHIMANO_CASE_STUDY/raw_layer/file_to_raw.ipynb


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/11 21:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Importing Dataset

In [3]:
brands_df,categories_df,customer_df,order_items_df,orders_df,products_df,staffs_df,stocks_df,stores_df = read_input_datasets()

#### Scenario Solving

##### Find the customer who has placed the highest total value of orders and calculate that value.

In [4]:
orders_df = orders_df.withColumnRenamed('customer_id','ord_customer_id')

customer_df\
    .join(orders_df,customer_df.customer_id==orders_df.ord_customer_id,how='inner')\
    .join(order_items_df,orders_df.order_id==order_items_df.order_id,how='inner')\
    .groupBy("customer_id","first_name","last_name")\
    .agg(sum("list_price").alias("total_value"))\
    .withColumn("drnk",dense_rank().over(Window.orderBy(col("total_value").desc())))\
    .filter(col("drnk")==1)\
    .show()

24/07/11 21:38:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 21:38:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 21:38:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 21:38:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 21:38:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 21:38:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/11 2

+-----------+----------+---------+------------------+----+
|customer_id|first_name|last_name|       total_value|drnk|
+-----------+----------+---------+------------------+----+
|          6|   Lyndsey|     Bean|24488.900000000005|   1|
+-----------+----------+---------+------------------+----+



##### List all products that have never been ordered along with their details.

In [7]:
order_items_df = order_items_df.withColumnRenamed('product_id','ord_it_product_id')
products_df\
    .join(order_items_df,products_df.product_id==order_items_df.ord_it_product_id,"left")\
    .filter(col("ord_it_product_id").isNull())\
    .select("product_id")\
    .show()

+----------+
|product_id|
+----------+
|         1|
|       121|
|       125|
|       154|
|       195|
|       267|
|       284|
|       291|
|       316|
|       317|
|       318|
|       319|
|       320|
|       321|
+----------+



##### Find the store with the highest average order value.